In [16]:
import numpy as np
import pandas as pd

raw_distance_matrix = np.load("./data/distances.npy")

/tmp/ipykernel_12459/1640478239.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
distance_matrix = raw_distance_matrix / raw_distance_matrix.max()
distance_matrix = distance_matrix[:10, :10]

In [4]:
n = len(distance_matrix)

In [5]:
# TODO: Maybe use jit for that
def get_surrounding_radius(selected_cities: np.ndarray):
    return distance_matrix[selected_cities].T[selected_cities].max() / 2

In [6]:
selected_cites = np.random.random(n) < 0.5

In [7]:
get_surrounding_radius(selected_cites)

0.3110310435295105

In [47]:
def initalize_random_state(n) :
    return np.random.random(n) < 0.5

In [43]:
from typing import Protocol
from functools import lru_cache


@lru_cache
def get_geometric_probs(n: int, p: int) -> np.ndarray:
    vector = np.array([p]) ** np.arange(1, n+1)
    return vector / np.sum(vector)


class MarkovChain(Protocol):
    @staticmethod
    def next_state(state: np.ndarray) -> np.ndarray:
        ...


class RandomWalk(MarkovChain):
    @staticmethod
    def next_state(state: np.ndarray) -> np.ndarray:
        u = np.random.random()
        if u < 1 / (n + 1):
            return state

        idx = np.random.randint(n)
        new_state = state.copy()
        new_state[idx] = ~new_state[idx]
        return new_state


class WeightedRandomJumps(MarkovChain):

    @staticmethod
    def next_state(state: np.ndarray) -> np.ndarray:
        n = len(state)
        probs = get_geometric_probs(n, 1/2)
        m = np.random.choice(np.arange(1, n+1), p=probs)
        idxs = np.random.choice(np.arange(n), m, replace=False)
        
        new_state = state.copy()
        new_state[idxs] = ~new_state[idxs]
        return new_state

In [46]:
def create_objective_function(beta: float = 1):
    def objective_function(state: np.ndarray):
        r = get_surrounding_radius(state)

        ...

In [48]:
class MetropolisHastings:

    def __init__(self, n_cities: int, n_workers: int, exploring_chain: MarkovChain, objective_function) -> None:
        self.exploring_chain = exploring_chain
        self.objective_function = objective_function

        self.states = [
            initalize_random_state(n_cities)
            for _ in range(n_workers)
        ]
        self.scores = [
            self.objective_function(state)
            for state in self.states
        ]

    def acceptance(self, prev_score, new_score):
        return np.min(1, np.exp(new_score - prev_score))

    def forward(self):
        for i, state in enumerate(self.states):
            new_state = self.exploring_chain.next_state(state)
            new_score = self.objective_function(new_state)
            u = np.random.random()
            if u <= self.acceptance(self.scores[i], new_score):
                self.states[i] = new_state
                self.scores = new_score


IndentationError: expected an indented block after function definition on line 14 (3655832837.py, line 16)